# **Data Profiling Notebook**

## **Initial Steps**

### Import the required libraries

In [ ]:
import pandas as pd
from numpy import log
from pandas import Series
from scipy.stats import norm, expon, lognorm
from seaborn import distplot, heatmap
from matplotlib.pyplot import show, subplots, figure, Axes, title
from utils.ds_charts import bar_chart, get_variable_types, choose_grid, multiple_bar_chart, multiple_line_chart, HEIGHT

### Read the data
Uncomment the data you want to analyse

In [ ]:
df = pd.read_csv("data/air_quality_tabular.csv", delimiter=",", na_values=None, parse_dates=["date"], infer_datetime_format=True) 
#df = pd.read_csv("data/air_quality_timeseries.csv", delimiter=",", na_values=None, parse_dates=["DATE"], infer_datetime_format=True)
#df = pd.read_csv("data/NYC_collisions_tabular.csv", delimiter=",", na_values=None, parse_dates={"CRASH_DATETIME": ["CRASH_DATE", "CRASH_TIME"]}, infer_datetime_format=True)
#df = pd.read_csv("data/NYC_collisions_timeseries.csv", delimiter=",", na_values=None, parse_dates=["timestamp"], infer_datetime_format=True)
df

## **Data Dimmensionality**

### Data shape

In [ ]:
df.shape

### Number of records vs number of variables

In [ ]:
values = {'Nr Records': df.shape[0], 'Nr Variables': df.shape[1]}
bar_chart(list(values.keys()), list(values.values()), title='Nr of Records vs Nr of Variables')
show()

### Variable types

In [ ]:
df.dtypes

### Convert object types to category type

In [ ]:
cat_vars = df.select_dtypes(include='object')
df[cat_vars.columns] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df.dtypes

### Number of variables per type

In [ ]:
variable_types = get_variable_types(df)
counts = {}
for tp in variable_types.keys():
    counts[tp] = len(variable_types[tp])
bar_chart(list(counts.keys()), list(counts.values()), title='Nr of Variables per Type')
show()

### Number of missing values per variable

In [ ]:
mv = {}
for var in df:
    nr = df[var].isna().sum()
    if nr > 0:
        mv[var] = nr
bar_chart(list(mv.keys()), list(mv.values()), title='Nr of Missing Values per Variable',
            xlabel='Variables', ylabel='Nr Missing Values', rotation=True)
show()

## **Data Distribution**

### Numeric variables summary

In [ ]:
df.describe()

### Global numeric variables boxplot

In [ ]:
df.boxplot(rot=45)
show()

### Single numeric variables boxplot

In [ ]:
numeric_vars = get_variable_types(df)['Numeric']
if [] == numeric_vars:
    raise ValueError('There are no numeric variables.')
rows, cols = choose_grid(len(numeric_vars))
fig, axs = subplots(rows, cols, figsize=(cols*HEIGHT, rows*HEIGHT), squeeze=False)
i, j = 0, 0
for n in range(len(numeric_vars)):
    axs[i, j].set_title('Boxplot for %s'%numeric_vars[n])
    axs[i, j].boxplot(df[numeric_vars[n]].dropna().values)
    i, j = (i + 1, 0) if (n+1) % cols == 0 else (i, j + 1)
show()

### Number of outliers per variable

In [ ]:
NR_STDEV: int = 2
numeric_vars = get_variable_types(df)['Numeric']
if [] == numeric_vars:
    raise ValueError('There are no numeric variables.')
outliers_iqr = []
outliers_stdev = []
summary5 = df.describe(include='number')
for var in numeric_vars:
    iqr = 1.5 * (summary5[var]['75%'] - summary5[var]['25%'])
    outliers_iqr += [
        df[df[var] > summary5[var]['75%']  + iqr].count()[var] +
        df[df[var] < summary5[var]['25%']  - iqr].count()[var]]
    std = NR_STDEV * summary5[var]['std']
    outliers_stdev += [
        df[df[var] > summary5[var]['mean'] + std].count()[var] +
        df[df[var] < summary5[var]['mean'] - std].count()[var]]
outliers = {'iqr': outliers_iqr, 'stdev': outliers_stdev}
figure(figsize=(12, HEIGHT))
multiple_bar_chart(numeric_vars, outliers, title='Nr of Outliers per Variable', xlabel='Variables', ylabel='Nr Outliers', percentage=False)
show()

### Basic variable distribution

In [ ]:
numeric_vars = get_variable_types(df)['Numeric']
if [] == numeric_vars:
    raise ValueError('There are no numeric variables.')
fig, axs = subplots(rows, cols, figsize=(cols*HEIGHT, rows*HEIGHT), squeeze=False)
i, j = 0, 0
for n in range(len(numeric_vars)):
    axs[i, j].set_title('Histogram for %s'%numeric_vars[n])
    axs[i, j].set_xlabel(numeric_vars[n])
    axs[i, j].set_ylabel("nr records")
    axs[i, j].hist(df[numeric_vars[n]].dropna().values, 'auto')
    i, j = (i + 1, 0) if (n+1) % cols == 0 else (i, j + 1)
show()

### Advanced variable distribution

In [ ]:
numeric_vars = get_variable_types(df)['Numeric']
if [] == numeric_vars:
    raise ValueError('There are no numeric variables.')
fig, axs = subplots(rows, cols, figsize=(cols*HEIGHT, rows*HEIGHT), squeeze=False)
i, j = 0, 0
for n in range(len(numeric_vars)):
    axs[i, j].set_title('Histogram with trend for %s'%numeric_vars[n])
    distplot(df[numeric_vars[n]].dropna().values, norm_hist=True, ax=axs[i, j], axlabel=numeric_vars[n])
    i, j = (i + 1, 0) if (n+1) % cols == 0 else (i, j + 1)
show()

### Normal, exponential and logarithmic distributions

In [ ]:
def compute_known_distributions(x_values: list) -> dict:
    distributions = dict()
    # Gaussian
    mean, sigma = norm.fit(x_values)
    distributions['Normal(%.1f,%.2f)'%(mean,sigma)] = norm.pdf(x_values, mean, sigma)
    # Exponential
    loc, scale = expon.fit(x_values)
    distributions['Exp(%.2f)'%(1/scale)] = expon.pdf(x_values, loc, scale)
    # LogNorm
    sigma, loc, scale = lognorm.fit(x_values)
    distributions['LogNor(%.1f,%.2f)'%(log(scale),sigma)] = lognorm.pdf(x_values, sigma, loc, scale)
    return distributions

def histogram_with_distributions(ax: Axes, series: Series, var: str):
    values = series.sort_values().values
    ax.hist(values, 20, density=True)
    distributions = compute_known_distributions(values)
    multiple_line_chart(values, distributions, ax=ax, title='Best fit for %s'%var, xlabel=var, ylabel='')

numeric_vars = get_variable_types(df)['Numeric']
if [] == numeric_vars:
    raise ValueError('There are no numeric variables.')
fig, axs = subplots(rows, cols, figsize=(cols*HEIGHT, rows*HEIGHT), squeeze=False)
i, j = 0, 0
for n in range(len(numeric_vars)):
    histogram_with_distributions(axs[i, j], df[numeric_vars[n]].dropna(), numeric_vars[n])
    i, j = (i + 1, 0) if (n+1) % cols == 0 else (i, j + 1)
show()

### Symbolic variables

In [ ]:
symbolic_vars = get_variable_types(df)['Symbolic']
if [] == symbolic_vars:
    raise ValueError('There are no symbolic variables.')

rows, cols = choose_grid(len(symbolic_vars))
fig, axs = subplots(rows, cols, figsize=(cols*HEIGHT, rows*HEIGHT), squeeze=False)
i, j = 0, 0
for n in range(len(symbolic_vars)):
    counts = df[symbolic_vars[n]].value_counts()
    bar_chart(counts.index.to_list(), counts.values, ax=axs[i, j], title='Histogram for %s'%symbolic_vars[n], xlabel=symbolic_vars[n], ylabel='nr records', percentage=False)
    i, j = (i + 1, 0) if (n+1) % cols == 0 else (i, j + 1)
show()

## **Data Granularity**

### Numeric values

In [ ]:
variables = get_variable_types(df)['Numeric']
if not variables:
    raise ValueError('There are no numeric variables.')
rows = len(variables)
bins = (10, 100, 1000)
cols = len(bins)
fig, axs = subplots(rows, cols, figsize=(cols * HEIGHT, rows * HEIGHT), squeeze=False)
for i in range(rows):
    for j in range(cols):
        axs[i, j].set_title('Histogram for %s %d bins' % (variables[i], bins[j]))
        axs[i, j].set_xlabel(variables[i])
        axs[i, j].set_ylabel('Nr Records')
        axs[i, j].hist(df[variables[i]].values, bins=bins[j])
show()

### Date values

In [ ]:
variables = get_variable_types(df)['Date']
if not variables:
    raise ValueError('There are no date variables.')
rows = len(variables)
bins = (10, 100, 1000)
cols = len(bins)
fig, axs = subplots(rows, cols, figsize=(cols * HEIGHT, rows * HEIGHT), squeeze=False)
for i in range(rows):
    for j in range(cols):
        axs[i, j].set_title('Histogram for %s %d bins' % (variables[i], bins[j]))
        axs[i, j].set_xlabel(variables[i])
        axs[i, j].set_ylabel('Nr records')
        axs[i, j].hist(df[variables[i]].values, bins=bins[j])
show()

## **Data Sparcity**

### Scatter-plots for numeric values

In [ ]:
numeric_vars = get_variable_types(df)['Numeric']
if [] == numeric_vars:
    raise ValueError('There are no numeric variables.')
rows, cols = len(numeric_vars)-1, len(numeric_vars)-1
fig, axs = subplots(rows, cols, figsize=(cols*HEIGHT, rows*HEIGHT), squeeze=False)
for i in range(len(numeric_vars)):
    var1 = numeric_vars[i]
    for j in range(i+1, len(numeric_vars)):
        var2 = numeric_vars[j]
        axs[i, j-1].set_title("%s x %s"%(var1,var2))
        axs[i, j-1].set_xlabel(var1)
        axs[i, j-1].set_ylabel(var2)
        axs[i, j-1].scatter(df[var1], df[var2])
show()

### Scatter-plots for symbolic values

In [ ]:
symbolic_vars = get_variable_types(df)['Symbolic']
if [] == symbolic_vars:
    raise ValueError('There are no symbolic variables.')
rows, cols = len(symbolic_vars)-1, len(symbolic_vars)-1
fig, axs = subplots(rows, cols, figsize=(cols*HEIGHT, rows*HEIGHT), squeeze=False)
for i in range(len(symbolic_vars)):
    var1 = symbolic_vars[i]
    for j in range(i+1, len(symbolic_vars)):
        var2 = symbolic_vars[j]
        axs[i, j-1].set_title("%s x %s"%(var1,var2))
        axs[i, j-1].set_xlabel(var1)
        axs[i, j-1].set_ylabel(var2)
        axs[i, j-1].scatter(df[var1], df[var2])
show()

### Correlation analysis

In [ ]:
corr_mtx = abs(df.corr())
print(corr_mtx)

### Correlation plot

In [ ]:
fig = figure(figsize=[12, 12])
heatmap(abs(corr_mtx), xticklabels=corr_mtx.columns, yticklabels=corr_mtx.columns, annot=True, cmap='Blues')
title('Correlation analysis')
show()